In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import os
import sys
import pandas as pd
import numpy as np
import quandl
import cvxpy as cvx

In [4]:
sys.path.insert(0, os.path.abspath('..'))
import helium

In [5]:
%matplotlib inline

In [6]:
quandl.ApiConfig.api_key = '8wnB7EcvJm1xEvJjzWBR'

In [7]:
tickers = ['AMZN', 'GOOGL', 'TSLA', 'NKE']
start_date='2012-01-01'
end_date='2016-12-31'
returns = pd.DataFrame(dict([(ticker, quandl.get('WIKI/'+ticker, 
                                    start_date=start_date, 
                                    end_date=end_date)['Adj. Close'].pct_change())
                for ticker in tickers]))
returns[["_CASH"]]=quandl.get('FRED/DTB3', start_date=start_date, end_date=end_date)/(250*100)
returns = returns.fillna(method='ffill').iloc[1:]

returns.tail()

,AMZN,GOOGL,NKE,TSLA,_CASH
Date,,,,,
2016-12-23,-0.007503,-0.002322,-0.004411,0.023459,0.000020
2016-12-27,0.014213,0.002637,-0.011944,0.029015,0.000020
2016-12-28,0.000946,-0.006618,-0.005264,0.000957,0.000021
2016-12-29,-0.009040,-0.002101,0.000784,-0.023027,0.000018
2016-12-30,-0.019970,-0.012991,-0.004505,-0.004612,0.000020


In [8]:
rets = returns.rolling(window=250, min_periods=250).mean().shift(1).dropna()
sigmas = returns.rolling(window=250, min_periods=250).cov().unstack().shift(1).stack()

rets.tail()

,AMZN,GOOGL,NKE,TSLA,_CASH
Date,,,,,
2016-12-23,0.000683,0.000219,-0.000660,-0.000074,0.000012
2016-12-27,0.000542,0.000150,-0.000706,-0.000124,0.000013
2016-12-28,0.000627,0.000179,-0.000691,-0.000023,0.000013
2016-12-29,0.000707,0.000215,-0.000664,-0.000052,0.000013
2016-12-30,0.000901,0.000302,-0.000598,0.000133,0.000013


In [9]:
cash_ticker = "_CASH"

In [10]:
price_sigma = rets.copy()
price_sigma[:] = 0.

In [11]:
volumes = pd.DataFrame(dict([(ticker, quandl.get('WIKI/'+ticker, 
                                    start_date=start_date, 
                                    end_date=end_date)['Adj. Volume'])
                for ticker in tickers]))
volumes[cash_ticker] = np.finfo(np.float).max

In [12]:
tcost_model=helium.TransactionCost(gamma = 1., 
                                   half_spread=10E-4,
                                   nonlin_coef = 0.,  #1.,
                                   sigmas = price_sigma, 
                                   nonlin_power = 1.5,
                                   volumes = volumes,
                                   asym_coef = 0.)

In [13]:
borrow_costs = rets.copy()
borrow_costs[:] = 1E-4
borrow_costs["_CASH"] = 0.

dividends = rets.copy()
dividends[:] = 0.


hcost_model=helium.HoldingCost(gamma = 1., borrow_costs=borrow_costs, dividends=dividends)

In [14]:
risk_model = helium.BasicRiskCost(5.0, sigmas)

In [15]:
leverage_limit = helium.LeverageLimitConstraint(3.)

In [16]:
holding_init = pd.Series(index=rets.columns, data=0.)
holding_init[cash_ticker] = 100.
holding_init

AMZN       0.0
GOOGL      0.0
NKE        0.0
TSLA       0.0
_CASH    100.0
dtype: float64

In [17]:
deltas = rets.copy()
deltas[:]  = 0.

In [18]:
r = helium.DefaultRet(rets=rets, deltas = deltas, gamma_decay=None)

In [19]:
spo_policy = helium.SinglePeriodOpt(rets=r,
                                   costs=[risk_model, hcost_model, tcost_model],
                                   constraints=[leverage_limit]
                                   )

In [20]:
simulator = helium.MarketSimulator(returns, volumes, costs=[hcost_model, tcost_model], cash_ticker='_CASH')

In [21]:
result = simulator.run(holding_init, policy=spo_policy, start_date='2013-01-03', end_date='2016-12-31')

INFO:root:Backtest started, from 2013-01-03 00:00:00 to 2016-12-30 00:00:00
INFO:root:Getting trades at date: 2013-01-03 00:00:00


NotImplementedError: 

In [ ]:
result.summary()

In [ ]:
result.v.plot(figsize=(12,5))

In [ ]:
result.w.plot(figsize=(12,6))